For each frame in the video is a node. Between every two nodes, there is a cost associated with
adding and removing apples. This cost is equal to the size of the exclusive or of the points in the two frames.
after calculating the cost between every two nodes, we can use the travelling salesman heuristic to find a path
through the frames that minimizes the number of apples added and removed.

In [46]:
import cv2
import numpy as np
from pathlib import Path
# add current directory to path
import sys
sys.path.append("~/src/badapple/badapple-irl/")
from python_tsp2.heuristics import solve_tsp_simulated_annealing, solve_tsp_lin_kernighan
from python_tsp2.exact import solve_tsp_dynamic_programming

In [3]:
# setup
badapple_path = Path(".").resolve().parent / "badapple-small.mp4"
square_side = 30
# skip to frame 364 because I already did every frame before that
frame_start = 365

width: 960, height: 720, fps: 30, n_frames: 6572


True

In [19]:
# get all the points in each frame
cap = cv2.VideoCapture(str(badapple_path))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"width: {width}, height: {height}, fps: {fps}, n_frames: {n_frames}")
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_start)

frame_count = 0
frame_points = []
original_frame_id_map = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if frame_count % 100 == 0:
        print(f"frame {frame_count}/{(n_frames-frame_start)//2}", end="\r")
    frame_count += 1

    current_frame_pixels = set()
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            section = frame[j: j + square_side, i: i + square_side]
            if (np.mean(section)) < 50:
                current_frame_pixels.add((i, j))
    frame_points.append(current_frame_pixels)
    original_frame_id_map[len(frame_points) - 1] = frame_start + (frame_count-1) * 2  # trust
    # skip a frame because I don't want to do every frame
    ret, frame = cap.read()
num_frames = len(frame_points)
print(f"\nnum_frames: {num_frames}")

width: 960, height: 720, fps: 30, n_frames: 6572
frame 3100/3103
num_frames: 3104


In [22]:
# calculate the cost between every two frames (distance matrix), and then run TSP algorithm to get optimized order of frames
costs = np.zeros((num_frames, num_frames))
for i in range(num_frames):
    for j in range(num_frames):
        costs[i][j] = len(frame_points[i] ^ frame_points[j])
        costs[j][i] = costs[i][j]

RecursionError: maximum recursion depth exceeded

In [47]:
import sys
import time
print(costs)
sys.setrecursionlimit(100000) 
start_time = time.time()
permutation, distance = solve_tsp_lin_kernighan(costs)
print(permutation)
print(f"Time taken: {time.time() - start_time}")
print("Distance: ", distance)


[[  0.  13.  32. ... 759. 759. 759.]
 [ 13.   0.  23. ... 746. 746. 746.]
 [ 32.  23.   0. ... 739. 739. 739.]
 ...
 [759. 746. 739. ...   0.   0.   0.]
 [759. 746. 739. ...   0.   0.   0.]
 [759. 746. 739. ...   0.   0.   0.]]
[0, 2338, 235, 233, 236, 231, 232, 230, 229, 228, 2, 439, 443, 440, 442, 441, 3, 444, 438, 437, 4, 5, 436, 445, 24, 20, 435, 446, 434, 447, 2442, 2430, 2441, 2440, 2439, 2438, 2437, 2436, 2435, 2431, 2434, 2433, 2432, 2212, 2358, 2360, 2359, 2361, 2362, 2363, 2364, 2365, 2367, 2368, 1745, 1744, 1743, 1742, 1741, 1740, 1739, 1738, 1737, 1736, 1735, 1734, 1733, 1732, 1731, 1730, 1729, 1728, 1727, 1726, 1725, 1724, 1723, 1722, 1721, 1720, 1719, 1718, 1717, 1714, 1715, 1716, 1713, 1712, 1711, 1710, 1709, 1708, 1707, 1706, 1705, 1704, 1703, 1702, 1701, 1700, 1699, 1698, 1697, 1696, 1695, 1694, 1693, 2746, 2747, 2748, 2749, 2750, 2752, 2753, 471, 2034, 881, 890, 483, 874, 2535, 3094, 2556, 1078, 107, 2532, 140, 145, 146, 193, 192, 191, 190, 189, 182, 178, 174, 175, 17

In [48]:
# calculate the number of apple moves using the normal order and optimized order of frames
base_apple_moves = 0
for i in range(num_frames - 1):
    base_apple_moves += len(frame_points[i] ^ frame_points[i + 1])
print(f"Base apple moves: {base_apple_moves}")
optimized_apple_moves = 0
for i in range(num_frames - 1):
    optimized_apple_moves += len(frame_points[permutation[i]] ^ frame_points[permutation[i + 1]])
print(f"Optimized apple moves: {optimized_apple_moves}, percentage of original: {optimized_apple_moves / base_apple_moves * 100}%")

Base apple moves: 127317
Optimized apple moves: 152975, percentage of original: 120.15284683113802%
